In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from layers import *

In [3]:
X = np.random.normal(size=[128, 32, 32, 16])

In [4]:
inputs = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 16])

In [5]:
def architecture(inputs, training):
    P = 0.5
    L = 54
    l = 1

    out = first_layer(inputs, training, 'input')

    with tf.variable_scope('stack1', reuse=tf.AUTO_REUSE):
        for i in range(1, 19):
            p = 1 - l/L * (1 - P)
            out = residual_block(out, 16, p, training, 'res'+str(i))
            l += 1

    with tf.variable_scope('stack2', reuse=tf.AUTO_REUSE):
        p = 1 - l/L * (1 - P)
        out = transition_block(out, 32, p, training, 'res'+str(1))
        l += 1

        for i in range(2, 19):
            p = 1 - l/L * (1 - P)
            out = residual_block(out, 32, p, training, 'res'+str(i))
            l += 1

    with tf.variable_scope('stack3', reuse=tf.AUTO_REUSE):
        p = 1 - l/L * (1 - P)
        out = transition_block(out, 64, p, training, 'res'+str(1))
        l += 1

        for i in range(2, 19):
            p = 1 - l/L * (1 - P)
            out = residual_block(out, 64, p, training, 'res'+str(i))

    return output_layer(out, 'out', 10)

In [6]:
out = architecture(inputs, True)
with tf.Session() as sess:
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(out, feed_dict={inputs: X})
    writer.close()

In [7]:
# with tf.Session() as sess:
#     ls = []
#     with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
#         for i in range(3):
#             out = architecture(inputs, True)
#             if i == 0:
#                 sess.run(tf.global_variables_initializer())
#             ls.append(sess.run(out, feed_dict={inputs: X}))